## This notebook is to load the annotated dataset and work with it.

Input: a EHR note (MIMIC)

Instruction: Please identify 5~10 word tokens from the EHR note. Those 5~10 word tokens should be most important for a patient to understand their clinical conditions, procedures, and treatment plans.

Output: keywords [use either human annotated, or just use MIMIC outputs
[overlapping keywords between notes and discharge instructions- advantage: llama and GPT4]

In [1]:
from utils import *

config = load_config()
projectPath = config.project_path
dataPath = config.get_data_path("annotated_examples_path")

# lfPath = dataPath.joinpath("LiverFailure")
notesPath = dataPath[0].joinpath("cancer")
rawannPath = dataPath[1].joinpath("rankings_300notes_20151217_cancer.txt")

# notesPath
dataPath[0], dataPath[1]

there are ...
['cancer', 'copd', 'diabetes', 'heart_failure', 'hypertension', 'liver_failure']


(PosixPath('/data/data_user/annotations/Jinying/notesAid_286notes/clean_raw_286notes'),
 PosixPath('/data/data_user/annotations/Jinying/notesAid_286notes/rawAnn'))

In [13]:
import pandas as pd

annotated_dicts = []
for datas in ['liver_failure', 'copd', 'hypertension', 'cancer', 'diabetes', 'heart_failure'] :
    path = dataPath[0].joinpath(datas)
    files = os.listdir(path)
    for file in files : 
        with open(path.joinpath(file), 'r') as f :
            text = f.read()
        
        annotated_dicts.append({"category" : datas, "noteid" : file, "text" : text})

notes = pd.DataFrame(annotated_dicts)
notes.to_pickle(Path(projectPath).joinpath("data/processed/notes.pkl"))

In [14]:
notes

,category,noteid,text
0,liver_failure,liver_failure.report37286.txt,This is a 50-year-old male with a history of d...
1,liver_failure,liver_failure.report37775.txt,Dr. name has discussed these results with you....
2,liver_failure,liver_failure.report38874.txt,"F/u on Osteoarthritis, chronic pain, HTN, Depr..."
3,liver_failure,liver_failure.report41972.txt,Very high a1c and glucose please follow up in ...
4,liver_failure,liver_failure.report51432.txt,name is a lovely just turned 65-year-old gentl...
...,...,...,...
281,heart_failure,heart_failure.report80980.txt,1. Multifactorial anemia secondary to both ren...
282,heart_failure,heart_failure.report85881.txt,The patient is being seen for an initial evalu...
283,heart_failure,heart_failure.report9402.txt,This 86-year-old coming in for a complete chec...
284,heart_failure,heart_failure.report94858.txt,The patient presents today for evaluation for ...


In [41]:
myList= []
for file in dataPath[1].iterdir() :
    myList.append(pd.read_csv(file, sep='\t'))
    
annotation_info_table = pd.concat(myList)

In [43]:
annotation_info_table['noteid'] = annotation_info_table.apply(lambda x : x['note'] + ".txt", axis=1)

In [47]:
reports = set(annotation_info_table['noteid'].unique())

Now filter ones with the reports

In [51]:
filtered_notes = notes[notes.noteid.isin(reports)]

============================================
- Now we save the processed notes and annotation information

In [53]:
filtered_notes.to_pickle("../data/processed/filtered_notes.pkl")
annotation_info_table.to_pickle("../data/processed/annotation_info_table.pkl")

## ======================================= Process annotation notes

In [2]:
import pandas as pd
pages = ['victoria', 'jiaping', 'jinying-3A', 'jinying-2B-part1', 'jinying_2B_part2','jinying_3C']

datas = []
for page in pages :
    df = pd.read_excel("../data/processed/all_20160531_changed.xlsx", page)
    df = df[['fileid', 'phrase']].copy()
    datas.append(df)

In [3]:
datas = pd.concat(datas, ignore_index=True).dropna()
datas = datas.reset_index(drop=True)

In [4]:
datas

,fileid,phrase
0,cancer.report7.txt,Follicular lymphoma
1,cancer.report7.txt,rituximab
2,cancer.report7.txt,grade I-II/III of the parotid gland and cervic...
3,cancer.report7.txt,hyperension
4,cancer.report7.txt,gout
...,...,...
2333,cancer.report27.txt,bilateral cerebral infarcts
2334,cancer.report27.txt,Keppra
2335,cancer.report27.txt,Bumex
2336,cancer.report27.txt,low-dose morphine drip


In [6]:
notes = pd.read_pickle("../data/processed/notes.pkl")
notes.head()

,category,noteid,text
0,liver_failure,liver_failure.report37286.txt,This is a 50-year-old male with a history of d...
1,liver_failure,liver_failure.report37775.txt,Dr. name has discussed these results with you....
2,liver_failure,liver_failure.report38874.txt,"F/u on Osteoarthritis, chronic pain, HTN, Depr..."
3,liver_failure,liver_failure.report41972.txt,Very high a1c and glucose please follow up in ...
4,liver_failure,liver_failure.report51432.txt,name is a lovely just turned 65-year-old gentl...


In [75]:
mergedData = notes.merge(combinedData)

In [77]:
mergedData.to_pickle("../data/processed/mergedData.pkl")

In [1]:
import pandas as pd
mergedData = pd.read_pickle("../data/processed/mergedData.pkl")

In [2]:
print(mergedData.shape)
mergedData.head()

(106, 4)


,category,noteid,text,Phrase
0,liver_failure,liver_failure.report37286.txt,This is a 50-year-old male with a history of d...,"Diarrhea-predominant, irritable bowel syndrome..."
1,liver_failure,liver_failure.report41972.txt,Very high a1c and glucose please follow up in ...,"a1c, diabetes, CLYCOHEMOGLOBIN A1C, HGBA1C,"
2,liver_failure,liver_failure.report51432.txt,name is a lovely just turned 65-year-old gentl...,"patellofemoral syndrome, physical therapy, Act..."
3,liver_failure,liver_failure.report55225.txt,name is a lovely 53-year-old gentleman who I h...,"Nonischemic cardiomyopathy, Persantine thalliu..."
4,liver_failure,liver_failure.report60517.txt,"NPOV \n\n67 year old male with HTN, Chronic Al...","HTN, Chronic Alcohol Abuse, remission, Chronic..."


In [7]:
print(mergedData['text'][0], "\n\n", mergedData['Phrase'][0])

This is a 50-year-old male with a history of diarrhea-predominant irritable bowel syndrome, who is coming in complaining of a one-month history of abdominal cramping in the epigastric region. The patient reports that his IBS was diagnosed two to three years ago when he presented with a history of alternating constipation and diarrhea post stressful events. He reports that his usual IBS flares last for 24-48 hours after a clear stressor and then spontaneously resolve. He reports that he had a sigmoidoscopy at that time, which was unremarkable, to rule out other pathologies such as IBD. He also has in the intervening time had a screening colonoscopy, where he underwent a polypectomy about two to three years ago, during which time he was also again found to not have any lesions consistent with inflammatory bowel disease. The patient reports that his IBS has been well controlled. About a month ago, he purchased a plane with the intent of renewing his pilot's license and has been stressed f

## ========================================== Process with Rankings

In [27]:
import pandas as pd
pages = ['victoria', 'jiaping', 'jinying-3A', 'jinying-2B-part1', 'jinying_2B_part2','jinying_3C']

datas = []
for page in pages :
    df = pd.read_excel("../data/raw/all_20160531.xlsx", page)
    # df = pd.read_excel("../data/processed/all_20160531_changed.xlsx", page)
    datas.append(df)

In [28]:
def preprocess_data(data, topn, ids = []) :
    data = data.dropna(subset=['ranking']).copy()
    # data = data.drop(columns = ['packet'])

    data['fileid'] = data['fileid'].str.lower()
    data['fileid'] = data['fileid'].str.strip()
    data['fileid'] = data.fileid.str.replace('heart failure', 'heart_failure')
    data['fileid'] = data.fileid.str.replace('liver failure', 'liver_failure')
    data['fileid'] = data['fileid'].str.replace(" ", "")
    data['fileid'] = data['fileid'].apply(lambda x : x + '.txt' if not x.endswith('.txt') else x)

    # process phrases
    data['phrase'] = data['phrase'].str.lower()
    data['phrase'] = data['phrase'].str.strip()

    # process colors 
    data['color'] = data['color'].str.lower()
    data['color'] = data['color'].replace({"yellow" : "y", "green" : "g"})

    data = data.dropna()

    # symptoms
    symptoms = data[data.color == 'y'].sort_values(by=['fileid', 'ranking']).groupby(['fileid','color']).head(topn)
    symptoms = symptoms.drop_duplicates(subset=['fileid', 'phrase'])

    # tests
    labtests = data[data.color == 'g'].sort_values(by=['fileid', 'ranking'])
    labtests = labtests[labtests.ranking < (topn + 1)].copy()

    result = pd.concat([symptoms, labtests], ignore_index=True)
    result = result.sort_values(by=['fileid','ranking'])

    # grouped = data.sort_values(by=['fileid','ranking'])
    # result = grouped.groupby(['fileid']).head(topn)

    result = result.drop_duplicates(subset=['fileid', 'ranking'])
    result = result[~result.fileid.isin(ids)].reset_index(drop=True)
    return result.reset_index(drop=True), result['fileid'].unique().tolist()

In [29]:
df = datas[3]
df[df.fileid == 'diabetes.report310966.txt']

,packet,fileid,phrase,color,ranking
223,2B,diabetes.report310966.txt,thrombocytosis,y,1.0
224,2B,diabetes.report310966.txt,Crohn disease,y,2.0
225,2B,diabetes.report310966.txt,budesonide,g,2.1
226,2B,diabetes.report310966.txt,diabetes mellitus,y,3.0
227,2B,diabetes.report310966.txt,metformin,g,3.1
228,2B,diabetes.report310966.txt,Iron deficiency,y,4.0
229,2B,diabetes.report310966.txt,oral iron,g,4.1
230,2B,diabetes.report310966.txt,myeloproliferative neoplasms,y,5.0


In [30]:
# don't erase this!
example_text = "cancer.report24.txt"
example_texts = ["cancer.report24.txt", "diabetes.report310966.txt"]

In [31]:
processed_datasets_top3 = []
ids = []

for data in datas :
    processed, data_ids = preprocess_data(data, 3, ids)
    ids.extend(data_ids)

    processed_datasets_top3.append(processed)

top3_dataset = pd.concat(processed_datasets_top3, ignore_index=True)
top3_dataset = top3_dataset[~top3_dataset.fileid.isin(example_texts)].reset_index(drop=True)


processed_datasets_top5 = []
ids = []
for data in datas :
    processed, data_ids = preprocess_data(data, 5, ids)
    ids.extend(data_ids)

    processed_datasets_top5.append(processed)

top5_dataset = pd.concat(processed_datasets_top5, ignore_index=True)
top5_dataset = top5_dataset[~top5_dataset.fileid.isin(example_texts)].reset_index(drop=True)


processed_datasets_top10 = []
ids = []
for data in datas :
    processed, data_ids = preprocess_data(data, 10, ids)
    ids.extend(data_ids)

    processed_datasets_top10.append(processed)
top10_dataset = pd.concat(processed_datasets_top10, ignore_index=True)
top10_dataset = top10_dataset[~top10_dataset.fileid.isin(example_texts)].reset_index(drop=True)

In [32]:
msk = top3_dataset.fileid != example_text
top3_dataset[msk]

,packet,fileid,phrase,color,ranking
0,1B,cancer.report28.txt,rheumatoi d arthritis,y,1.0
1,1B,cancer.report28.txt,rituximan,g,1.1
2,1B,cancer.report28.txt,denosumab,g,2.1
3,1B,cancer.report28.txt,hypercalcemic,g,2.2
4,1B,cancer.report37.txt,extranodal indolent marginal zone lymphoma,y,1.0
...,...,...,...,...,...
654,3C,cancer.report3.txt,bactrim prophylax,g,1.6
655,3C,cancer.report3.txt,midgut volvulus,y,2.0
656,3C,cancer.report3.txt,malrotation,g,2.1
657,3C,cancer.report3.txt,dystonic reaction,y,3.0


In [33]:
example_text = 'cancer.report24.txt'
top3_dataset[top3_dataset.fileid == example_text]

,packet,fileid,phrase,color,ranking


Now merge with notes

In [34]:
import pandas as pd

notes = pd.read_pickle("../data/processed/notes.pkl")
notes

,category,noteid,text
0,liver_failure,liver_failure.report37286.txt,This is a 50-year-old male with a history of d...
1,liver_failure,liver_failure.report37775.txt,Dr. name has discussed these results with you....
2,liver_failure,liver_failure.report38874.txt,"F/u on Osteoarthritis, chronic pain, HTN, Depr..."
3,liver_failure,liver_failure.report41972.txt,Very high a1c and glucose please follow up in ...
4,liver_failure,liver_failure.report51432.txt,name is a lovely just turned 65-year-old gentl...
...,...,...,...
281,heart_failure,heart_failure.report80980.txt,1. Multifactorial anemia secondary to both ren...
282,heart_failure,heart_failure.report85881.txt,The patient is being seen for an initial evalu...
283,heart_failure,heart_failure.report9402.txt,This 86-year-old coming in for a complete chec...
284,heart_failure,heart_failure.report94858.txt,The patient presents today for evaluation for ...


In [35]:
print(notes[notes.noteid == 'diabetes.report310966.txt']['text'][205])

name name is a 58-year-old man referred for evaluation of thrombocytosis. Prior CBCs from xxxx through xxxx revealed platelet counts ranging from 400,000 to 500,000, but no more recent studies are available. He has long-standing Crohn disease and although he says he has not had gastrointestinal bleeding in the past, he has been given iron, which he is taking twice daily. He has black stool, but notes no blood and he has not had hematemesis. He notes no blood in his urine or sputum and he has no epistaxis. He discontinued the use of iron yesterday because he thought that might alleviate his gastrointestinal complaints, but he does not feel different today. He is cared for by Dr. name name at St. hospital Medical Center in location. He has no history of prior cancers, tuberculosis or other infectious diseases. He has been taking budesonide for his Crohn disease. He has no unexplained fevers, although he states he often feels hot. He has no soaking sweats and has not had unexplained weigh

In [36]:
notes['noteid'].nunique()

286

In [39]:
import pickle

# the data are 105 dataset
file_top3 = top3_dataset['fileid'].unique()
# file_top5 = top3_dataset['fileid'].unique()

filtered_notes = notes[notes.noteid.isin(file_top3)].reset_index(drop=True)
filtered_notes

processed_ranking_datasets = (top3_dataset, top5_dataset, top10_dataset, filtered_notes)
with open("../data/processed/processed_ranking_datasets.pkl", 'wb') as f :
    pickle.dump(processed_ranking_datasets, f)

In [38]:
print(notes[~notes.noteid.isin(example_texts)]['text'].values[0])

This is a 50-year-old male with a history of diarrhea-predominant irritable bowel syndrome, who is coming in complaining of a one-month history of abdominal cramping in the epigastric region. The patient reports that his IBS was diagnosed two to three years ago when he presented with a history of alternating constipation and diarrhea post stressful events. He reports that his usual IBS flares last for 24-48 hours after a clear stressor and then spontaneously resolve. He reports that he had a sigmoidoscopy at that time, which was unremarkable, to rule out other pathologies such as IBD. He also has in the intervening time had a screening colonoscopy, where he underwent a polypectomy about two to three years ago, during which time he was also again found to not have any lesions consistent with inflammatory bowel disease. The patient reports that his IBS has been well controlled. About a month ago, he purchased a plane with the intent of renewing his pilot's license and has been stressed f